In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, SimpleRNN, Dense
from tensorflow.keras.models import Model


Electrodes:Fp1,Fp2,F7,F3,Fz,F4,F8,FC5,FC1,FC2,FC6,T7,C3,Cz,C4,T8,TP9,CP5,CP1,CP2,CP6,TP10,P7,P3,Pz,P4,P8,PO9,O1,Oz,O2,PO10


In [3]:

electrodes_list = ['Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8','FC5','FC1','FC2','FC6','T7','C3','Cz','C4','T8','TP9','CP5','CP1','CP2','CP6','TP10','P7','P3','Pz','P4','P8','PO9','O1','Oz','O2','PO10']
electrodes = {'Fp1':[],'Fp2':[],'F7':[],'F3':[],'Fz':[],'F4':[],'F8':[],'FC5':[],'FC1':[],'FC2':[],'FC6':[],'T7':[],'C3':[],'Cz':[],'C4':[],'T8':[],'TP9':[],'CP5':[],'CP1':[],'CP2':[],'CP6':[],'TP10':[],'P7':[],'P3':[],'Pz':[],'P4':[],'P8':[],'PO9':[],'O1':[],'Oz':[],'O2':[],'PO10':[]}
averages =  {'Fp1':[],'Fp2':[],'F7':[],'F3':[],'Fz':[],'F4':[],'F8':[],'FC5':[],'FC1':[],'FC2':[],'FC6':[],'T7':[],'C3':[],'Cz':[],'C4':[],'T8':[],'TP9':[],'CP5':[],'CP1':[],'CP2':[],'CP6':[],'TP10':[],'P7':[],'P3':[],'Pz':[],'P4':[],'P8':[],'PO9':[],'O1':[],'Oz':[],'O2':[],'PO10':[]}
electrodes_copy = {'Fp1':[],'Fp2':[],'F7':[],'F3':[],'Fz':[],'F4':[],'F8':[],'FC5':[],'FC1':[],'FC2':[],'FC6':[],'T7':[],'C3':[],'Cz':[],'C4':[],'T8':[],'TP9':[],'CP5':[],'CP1':[],'CP2':[],'CP6':[],'TP10':[],'P7':[],'P3':[],'Pz':[],'P4':[],'P8':[],'PO9':[],'O1':[],'Oz':[],'O2':[],'PO10':[]}
averages_copy =  {'Fp1':[],'Fp2':[],'F7':[],'F3':[],'Fz':[],'F4':[],'F8':[],'FC5':[],'FC1':[],'FC2':[],'FC6':[],'T7':[],'C3':[],'Cz':[],'C4':[],'T8':[],'TP9':[],'CP5':[],'CP1':[],'CP2':[],'CP6':[],'TP10':[],'P7':[],'P3':[],'Pz':[],'P4':[],'P8':[],'PO9':[],'O1':[],'Oz':[],'O2':[],'PO10':[]}


In [37]:
#load data
spr_data = pd.read_csv('dbc_spr_data_combined.csv')
data = pd.read_csv("dbc_data.csv")
    

In [38]:
data

,ItemNum,Condition,Subject,Timestamp,TrialNum,Fp1,Fp2,F7,F3,Fz,...,P8,PO9,O1,Oz,O2,PO10,VEOG,HEOG,Assoc,Plaus
0,1,control,5,-200,197,-1.20971,2.52400,-5.24529,-3.35825,-5.09060,...,2.29405,0.64024,-4.89769,-6.64080,-3.25357,-0.07328,0.71351,1.91076,6.8,6.416667
1,1,control,5,-198,197,-0.83101,2.92082,-5.41510,-3.22667,-4.61091,...,2.45947,0.78223,-5.05034,-6.61679,-2.91444,0.10294,0.67929,2.78725,6.8,6.416667
2,1,control,5,-196,197,-0.11259,3.49500,-4.92985,-2.57258,-3.57396,...,2.82271,1.14153,-4.55636,-5.89756,-2.12281,0.54867,0.59286,3.42723,6.8,6.416667
3,1,control,5,-194,197,0.89073,4.20807,-3.93422,-1.44027,-2.04021,...,3.32898,1.65022,-3.49408,-4.58794,-0.94843,1.23226,0.41795,3.69663,6.8,6.416667
4,1,control,5,-192,197,2.05291,4.89957,-2.72258,0.02966,-0.17849,...,3.87386,2.19716,-2.01769,-2.85254,0.51508,2.09137,0.10579,3.52454,6.8,6.416667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1293595,90,script-unrelated,21,1190,1588,0.73032,-0.50984,-1.14073,-10.83687,-13.13879,...,0.50029,-16.89443,-1.90192,1.55856,2.53320,12.71346,-29.60788,13.07202,3.9,2.500000
1293596,90,script-unrelated,21,1192,1588,2.53046,0.63507,0.42298,-9.91345,-13.02235,...,-2.11160,-14.96593,-2.05659,1.22516,1.16928,14.58646,-29.55238,11.50487,3.9,2.500000
1293597,90,script-unrelated,21,1194,1588,4.39755,1.99197,1.90782,-8.98467,-12.78465,...,-4.65342,-13.06380,-2.22250,0.89966,-0.03291,16.60000,-29.66380,9.95807,3.9,2.500000
1293598,90,script-unrelated,21,1196,1588,6.28774,3.58098,3.27531,-8.00595,-12.30128,...,-6.98318,-11.33832,-2.37400,0.60154,-1.09776,18.68834,-30.02666,8.60108,3.9,2.500000


In [6]:
spr_data.head(5)

,ItemNum,Condition,Subject,RT,Assoc,Plaus,Accuracy
0,1,control,19,841,6.8,6.416667,1
1,1,control,10,754,6.8,6.416667,1
2,1,control,13,1197,6.8,6.416667,1
3,1,control,16,351,6.8,6.416667,1
4,1,control,7,1051,6.8,6.416667,1


In [7]:
#copy of EEG data
data_copy = data.loc[:]
data_copy

,ItemNum,Condition,Subject,Timestamp,TrialNum,Fp1,Fp2,F7,F3,Fz,...,P8,PO9,O1,Oz,O2,PO10,VEOG,HEOG,Assoc,Plaus
0,1,control,5,-200,197,-1.20971,2.52400,-5.24529,-3.35825,-5.09060,...,2.29405,0.64024,-4.89769,-6.64080,-3.25357,-0.07328,0.71351,1.91076,6.8,6.416667
1,1,control,5,-198,197,-0.83101,2.92082,-5.41510,-3.22667,-4.61091,...,2.45947,0.78223,-5.05034,-6.61679,-2.91444,0.10294,0.67929,2.78725,6.8,6.416667
2,1,control,5,-196,197,-0.11259,3.49500,-4.92985,-2.57258,-3.57396,...,2.82271,1.14153,-4.55636,-5.89756,-2.12281,0.54867,0.59286,3.42723,6.8,6.416667
3,1,control,5,-194,197,0.89073,4.20807,-3.93422,-1.44027,-2.04021,...,3.32898,1.65022,-3.49408,-4.58794,-0.94843,1.23226,0.41795,3.69663,6.8,6.416667
4,1,control,5,-192,197,2.05291,4.89957,-2.72258,0.02966,-0.17849,...,3.87386,2.19716,-2.01769,-2.85254,0.51508,2.09137,0.10579,3.52454,6.8,6.416667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1293595,90,script-unrelated,21,1190,1588,0.73032,-0.50984,-1.14073,-10.83687,-13.13879,...,0.50029,-16.89443,-1.90192,1.55856,2.53320,12.71346,-29.60788,13.07202,3.9,2.500000
1293596,90,script-unrelated,21,1192,1588,2.53046,0.63507,0.42298,-9.91345,-13.02235,...,-2.11160,-14.96593,-2.05659,1.22516,1.16928,14.58646,-29.55238,11.50487,3.9,2.500000
1293597,90,script-unrelated,21,1194,1588,4.39755,1.99197,1.90782,-8.98467,-12.78465,...,-4.65342,-13.06380,-2.22250,0.89966,-0.03291,16.60000,-29.66380,9.95807,3.9,2.500000
1293598,90,script-unrelated,21,1196,1588,6.28774,3.58098,3.27531,-8.00595,-12.30128,...,-6.98318,-11.33832,-2.37400,0.60154,-1.09776,18.68834,-30.02666,8.60108,3.9,2.500000


In [8]:
#index by by item number, subject and condition so that its easy to pull one EEG experiment later
data_copy =data_copy.set_index(['ItemNum', 'Subject', 'Condition'])

In [9]:
#drop columns that are not necessary
data_copy = data_copy.drop(['TrialNum', 'HEOG', 'VEOG', 'Assoc', 'Plaus'], axis = 1)

In [10]:
data_copy

Timestamp      Fp1      Fp2       F7   
ItemNum Subject Condition                                                
1       5       control                -200 -1.20971  2.52400 -5.24529  \
                control                -198 -0.83101  2.92082 -5.41510   
                control                -196 -0.11259  3.49500 -4.92985   
                control                -194  0.89073  4.20807 -3.93422   
                control                -192  2.05291  4.89957 -2.72258   
...                                     ...      ...      ...      ...   
90      21      script-unrelated       1190  0.73032 -0.50984 -1.14073   
                script-unrelated       1192  2.53046  0.63507  0.42298   
                script-unrelated       1194  4.39755  1.99197  1.90782   
                script-unrelated       1196  6.28774  3.58098  3.27531   
                script-unrelated       1198  8.17561  5.38782  4.52047   

                                        F3        Fz       F4       F8   
ItemNum Subject Condition                                                
1       5       control           -3.35825  -5.09060 -5.03072  0.75808  \
                control           -3.22667  -4.61091 -4.05609  0.93607   
                control           -2.57258  -3.57396 -2.35894  1.21045   
                control           -1.44027  -2.04021 -0.03051  1.55044   
                control            0.02966  -0.17849  2.65758  1.95099   
...                                    ...       ...      ...      ...   
90      21      script-unrelated -10.83687 -13.13879 -5.03900  1.34555   
                script-unrelated  -9.91345 -13.02235 -4.77273  1.19406   
                script-unrelated  -8.98467 -12.78465 -4.23679  1.34813   
                script-unrelated  -8.00595 -12.30128 -3.37068  1.87702   
                script-unrelated  -6.93121 -11.47295 -2.14472  2.79418   

                                      FC5      FC1  ...        P7       P3   
ItemNum Subject Condition                           ...                      
1       5       control          -5.14854 -4.76760  ...  -3.01284 -2.09255  \
                control          -4.88549 -4.65950  ...  -3.43947 -2.32634   
                control          -4.19749 -3.98448  ...  -3.37743 -2.03314   
                control          -3.14670 -2.79818  ...  -2.86173 -1.27626   
                control          -1.85683 -1.26480  ...  -2.01307 -0.17923   
...                                   ...      ...  ...       ...      ...   
90      21      script-unrelated -5.72982 -7.95168  ... -11.74718 -1.86121   
                script-unrelated -4.65782 -7.13739  ... -11.66094 -1.13657   
                script-unrelated -3.63225 -6.36322  ... -11.49469 -0.47284   
                script-unrelated -2.62036 -5.56882  ... -11.22251  0.10544   
                script-unrelated -1.57638 -4.69880  ... -10.78656  0.62664   

                                       Pz       P4       P8       PO9   
ItemNum Subject Condition                                               
1       5       control          -0.28447  0.69816  2.29405   0.64024  \
                control          -0.14533  1.02523  2.45947   0.78223   
                control           0.51351  1.76842  2.82271   1.14153   
                control           1.62937  2.85362  3.32898   1.65022   
                control           3.05988  4.12485  3.87386   2.19716   
...                                   ...      ...      ...       ...   
90      21      script-unrelated -0.62735 -2.35254  0.50029 -16.89443   
                script-unrelated -1.00181 -3.97723 -2.11160 -14.96593   
                script-unrelated -1.33264 -5.42567 -4.65342 -13.06380   
                script-unrelated -1.59978 -6.60070 -6.98318 -11.33832   
                script-unrelated -1.77155 -7.44158 -8.96632  -9.88049   

                                       O1       Oz       O2      PO10  
ItemNum Subject Condition                                              

In [11]:
#testing accesing an experiment from the dataframe
test = data_copy.loc[(1,5, 'control')]
test


C:\Users\crist\AppData\Local\Temp\ipykernel_31752\2445720375.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  test = data_copy.loc[(1,5, 'control')]


Timestamp       Fp1      Fp2       F7        F3   
ItemNum Subject Condition                                                    
1       5       control         -200  -1.20971  2.52400 -5.24529  -3.35825  \
                control         -198  -0.83101  2.92082 -5.41510  -3.22667   
                control         -196  -0.11259  3.49500 -4.92985  -2.57258   
                control         -194   0.89073  4.20807 -3.93422  -1.44027   
                control         -192   2.05291  4.89957 -2.72258   0.02966   
...                              ...       ...      ...      ...       ...   
                control         1190 -13.30694 -6.29164 -6.31052 -10.81505   
                control         1192 -11.90899 -4.91423 -5.79533 -11.12276   
                control         1194 -11.35194 -4.31777 -5.72996 -11.56740   
                control         1196 -11.68057 -4.45219 -6.13633 -12.12251   
                control         1198 -12.73966 -5.03117 -6.85307 -12.70885   

                                Fz       F4        F8      FC5      FC1  ...   
ItemNum Subject Condition                                                ...   
1       5       control   -5.09060 -5.03072   0.75808 -5.14854 -4.76760  ...  \
                control   -4.61091 -4.05609   0.93607 -4.88549 -4.65950  ...   
                control   -3.57396 -2.35894   1.21045 -4.19749 -3.98448  ...   
                control   -2.04021 -0.03051   1.55044 -3.14670 -2.79818  ...   
                control   -0.17849  2.65758   1.95099 -1.85683 -1.26480  ...   
...                            ...      ...       ...      ...      ...  ...   
                control   -5.86176 -9.21278 -14.90361 -4.98216  7.53338  ...   
                control   -5.53842 -8.96457 -13.26767 -5.00517  7.61398  ...   
                control   -5.45285 -9.06355 -11.96487 -5.22972  7.58312  ...   
                control   -5.60692 -9.41442 -11.08642 -5.64043  7.45572  ...   
                control   -5.91319 -9.88619 -10.64151 -6.15370  7.30669  ...   

                                P7        P3        Pz        P4       P8   
ItemNum Subject Condition                                                   
1       5       control   -3.01284  -2.09255  -0.28447   0.69816  2.29405  \
                control   -3.43947  -2.32634  -0.14533   1.02523  2.45947   
                control   -3.37743  -2.03314   0.51351   1.76842  2.82271   
                control   -2.86173  -1.27626   1.62937   2.85362  3.32898   
                control   -2.01307  -0.17923   3.05988   4.12485  3.87386   
...                            ...       ...       ...       ...      ...   
                control    6.16513  18.56332  10.83941  15.37572 -9.63348   
                control    5.67340  18.28190  10.77579  15.75905 -9.12688   
                control    5.13027  17.90837  10.61339  16.06617 -8.68536   
                control    4.60223  17.47593  10.36430  16.27186 -8.31606   
                control    4.18501  17.08104  10.10966  16.43237 -7.93379   

                               PO9       O1       Oz       O2      PO10  
ItemNum Subject Condition                                                
1       5       control    0.64024 -4.89769 -6.64080 -3.25357  -0.07328  
                control    0.78223 -5.05034 -6.61679 -2.91444   0.10294  
                control    1.14153 -4.55636 -5.89756 -2.12281   0.54867  
                control    1.65022 -3.49408 -4.58794 -0.94843   1.23226  
                control    2.19716 -2.01769 -2.85254  0.51508   2.09137  
...                            ...      ...      ...      ...       ...  
                control   -7.26146 -0.23115  9.30324  0.16624 -11.49276  
                control   -6.88687 -0.75502  8.95166  0.24435  -9.63563  
                control   -6.91096 -1.38896  8.43272  0.08944  -8.87305  
                control   -7.30122 -2.05450  7.84270 -0.20200  -9.16813  
                control   -7.91904 -2.61992  7.33267 -0.44492 -10.32064  

[700 r

In [15]:
#list of unique experiments 
uniques = list(dict.fromkeys(data_copy.index.values.tolist()))
uniques

[(1, 5, 'control'),
 (1, 8, 'control'),
 (1, 14, 'control'),
 (1, 17, 'control'),
 (1, 20, 'control'),
 (1, 23, 'control'),
 (1, 26, 'control'),
 (1, 2, 'script-related'),
 (1, 6, 'script-related'),
 (1, 9, 'script-related'),
 (1, 12, 'script-related'),
 (1, 15, 'script-related'),
 (1, 18, 'script-related'),
 (1, 21, 'script-related'),
 (1, 3, 'script-unrelated'),
 (1, 7, 'script-unrelated'),
 (1, 10, 'script-unrelated'),
 (1, 13, 'script-unrelated'),
 (1, 16, 'script-unrelated'),
 (1, 19, 'script-unrelated'),
 (1, 25, 'script-unrelated'),
 (2, 2, 'control'),
 (2, 6, 'control'),
 (2, 9, 'control'),
 (2, 12, 'control'),
 (2, 15, 'control'),
 (2, 18, 'control'),
 (2, 21, 'control'),
 (2, 3, 'script-related'),
 (2, 7, 'script-related'),
 (2, 10, 'script-related'),
 (2, 13, 'script-related'),
 (2, 16, 'script-related'),
 (2, 19, 'script-related'),
 (2, 25, 'script-related'),
 (2, 5, 'script-unrelated'),
 (2, 8, 'script-unrelated'),
 (2, 14, 'script-unrelated'),
 (2, 17, 'script-unrelated')

In [17]:
#dictionary of all experiments;
#the format of the key is a tuple of the form (ItemNum, Subject, Condition)
subject_data = dict()
for item in uniques:
    subject_data[item] = data_copy.loc[item]
    

C:\Users\crist\AppData\Local\Temp\ipykernel_31752\1548147947.py:3: PerformanceWarning: indexing past lexsort depth may impact performance.
  subject_data[item] = data_copy.loc[item]


In [35]:
#separate dictionary of averages of all electrodes per experiment
#same format
subject_data_avg = dict()
for item in subject_data:
    subject_data_avg[item] = subject_data[item].mean()
    subject_data_avg[item] = subject_data_avg[item].drop('Timestamp')

In [36]:
subject_data_avg[(1,5,'control')]

Fp1     -3.283027
Fp2      1.794766
F7       0.469282
F3      -0.386121
Fz       3.035667
F4       0.111679
F8      -1.333367
FC5      1.598490
FC1     10.411323
FC2     12.241962
FC6     -5.330903
T7      -4.594679
C3       6.767656
Cz       8.606985
C4      20.344406
T8      -3.905636
TP9     -6.237751
CP5      4.255055
CP1     12.820719
CP2     10.509449
CP6      5.118414
TP10    -3.030997
P7       3.468283
P3      12.183570
Pz       9.895105
P4      13.754308
P8       1.564951
PO9     -3.275927
O1       1.531946
Oz       6.417109
O2       3.606478
PO10    -4.752072
dtype: float64

In [16]:
#The P600 as a continuous index of integration effort